In [ ]:
!pip install --upgrade tensorflow==1.15

Requirement already up-to-date: tensorflow==1.15 in /usr/local/lib/python3.7/dist-packages (1.15.0)


In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler # bisa MinMaxScaler ato StandardScaler

In [ ]:
df = pd.read_csv('winequality-white.csv', sep=';')

In [ ]:
cols = df.columns
features = df[cols[0:-1]].values
label = df[cols[-1]].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.1)

In [ ]:
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [ ]:
encoder = OneHotEncoder().fit(y_train)
y_train = encoder.transform(y_train).toarray()
y_test = encoder.transform(y_test).toarray()

In [ ]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
y_train, y_test = y_train.astype('float32'), y_test.astype('float32')

In [ ]:
EPOCH = 1000
ALPHA = 0.01
n_features = x_train.shape[1]
n_classes = y_train.shape[1]

In [ ]:
input_tensor = tf.placeholder(tf.float32)
label_tensor = tf.placeholder(tf.float32)

In [ ]:
neurons = [64, 64, n_classes]
n_layers = len(neurons)

In [ ]:
parameters = {}
for i in range(n_layers):
  parameters[f'W{i+1}'] = tf.Variable(
      tf.random.normal([
          n_features if (i == 0) else neurons[i-1], neurons[i]
      ]),
      tf.float32
  )

  parameters[f'B{i+1}'] = tf.Variable(
      tf.random.normal(
          [1, neurons[i]]
      ),
      tf.float32
  )

In [ ]:
# parameters = {
#     'W1' : tf.Variable(
#         tf.random.normal([
#             n_features, 64
#         ]),
#         tf.float32
#     ),
#     'B1' : tf.Variable(
#         tf.random.normal([
#             1, 64
#         ]),
#         tf.float32
#     ),
#     'W2' : tf.Variable(
#         tf.random.normal([
#             64, 64
#         ]),
#         tf.float32
#     ),
#     'B2' : tf.Variable(
#         tf.random.normal([
#             1, 64
#         ]),
#         tf.float32
#     ),
#     'W3' : tf.Variable(
#         tf.random.normal([
#             64, 7
#         ]),
#         tf.float32
#     ),
#     'B3' : tf.Variable(
#         tf.random.normal([
#             1, 7
#         ]),
#         tf.float32
#     ),
# }

In [ ]:
def forward():
  # z1 = tf.matmul(input_tensor, parameters['W1']) + parameters['B1']
  # a1 = tf.nn.tanh(z1)

  # z2 = tf.matmul(a1, parameter['W2']) + parameters['B2']
  # a2 = tf.nn.tanh(z2)

  # z3 = tf.matmul(a2, parameter['W3']) + parameters['B3']
  # a3 = tf.nn.softmax(z3)

  a = input_tensor
  for i in range(n_layers):
    w = parameters[f'W{i+1}']
    b = parameters[f'B{i+1}']

    z = tf.matmul(a, w) + b
    if i == n_layers - 1:
      a = tf.nn.softmax(z)
    else:
      a = tf.nn.tanh(z)
  return a

In [ ]:
with tf.Session() as sess:
  logits_tensor = forward()
  loss_tensor = tf.reduce_mean(0.5 * (label_tensor - logits_tensor) ** 2)

  true_preds_tensor = tf.equal(
      tf.argmax(logits_tensor, axis=1), tf.argmax(label_tensor, axis=1))
  acc_tensor = tf.reduce_mean(tf.cast(true_preds_tensor, tf.float32))

  optimizer = tf.train.AdamOptimizer(ALPHA).minimize(loss_tensor)

  sess.run(tf.global_variables_initializer())

  for i in range(EPOCH):
    sess.run(optimizer, feed_dict={
        input_tensor:x_train,
        label_tensor:y_train
    })
    loss = sess.run(loss_tensor, feed_dict={
        input_tensor:x_train,
        label_tensor:y_train
    })
    acc = sess.run(acc_tensor, feed_dict={
        input_tensor:x_train,
        label_tensor:y_train
    })
    val_loss = sess.run(loss_tensor, feed_dict={
        input_tensor:x_test,
        label_tensor:y_test
    })
    val_acc = sess.run(acc_tensor, feed_dict={
        input_tensor:x_test,
        label_tensor:y_test
    })
    print(f'Epoch {i+1} - Loss {loss:.4f} - Accuracy {(acc*100):.2f} % - Val Loss {val_loss:.4f} - Val Accuracy {(val_acc * 100):.2f} %')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1 - Loss 0.1098 - Accuracy 13.54 % - Val Loss 0.1127 - Val Accuracy 12.86 %
Epoch 2 - Loss 0.1057 - Accuracy 16.38 % - Val Loss 0.1090 - Val Accuracy 14.69 %
Epoch 3 - Loss 0.1019 - Accuracy 19.31 % - Val Loss 0.1043 - Val Accuracy 17.76 %
Epoch 4 - Loss 0.0987 - Accuracy 21.76 % - Val Loss 0.0997 - Val Accuracy 22.04 %
Epoch 5 - Loss 0.0956 - Accuracy 24.32 % - Val Loss 0.0963 - Val Accuracy 24.69 %
Epoch 6 - Loss 0.0923 - Accuracy 26.75 % - Val Loss 0.0932 - Val Accuracy 26.12 %
Epoch 7 - Loss 0.0892 - Accuracy 29.24 % - Val Loss 0.0907 - Val Accuracy 27.96 %
Epoch 8 - Loss 0.0870 - Accuracy 31.42 % - Val Loss 0.0888 - Val Accuracy 29.59 %
Epoch 9 - Loss 0.0848 - Accuracy 33.21 % - Val Loss 0.0867 - Val Accuracy 31.63 %
Epoch 10 - Loss 0.0828 - Accuracy 35.16 % - Val Loss 0.0852 - Val Accuracy 33.47 %
Epoch 11 - Loss 0.0810 - Accuracy 36.96 % - Val Loss 0.0834 - Val Accuracy 34.29 %
Ep